# ASP.NET Core with Multiple HttpClients in Jupyter Lab


This notebook demonstrates how to run an **ASP.NET Core app** with multiple HttpClient configurations in Jupyter Lab using the **.NET Interactive kernel**.



In [ ]:
#!csharp
#load "../HttpLoggerHelper.csx"
#r "nuget: Duende.AccessTokenManagement, 3.2.0"
#r "nuget: Microsoft.Extensions.Hosting, 9.0.5"
#r "nuget: Microsoft.Extensions.Caching.Memory, 9.0.0"
#r "nuget: Fhi.Authentication.Extensions, 1.0.0"

using Microsoft.Extensions.Hosting;
using Duende.AccessTokenManagement;
using Duende.IdentityModel;
using Duende.IdentityModel.Client;
using Fhi.Authentication.Tokens;
using Microsoft.Extensions.DependencyInjection;
using System.Net.Http;

public static class StartupWithMultipleHttpClient
{
    public static WebApplication ConfigureServices(this WebApplicationBuilder builder)
    {
        builder.Configuration.AddJsonFile("appsettings.multipleclient.json", optional: false, reloadOnChange: true);
        builder.Services.Configure<List<HttpClientConfiguration>>(
            builder.Configuration.GetSection("HttpClientConfigurations"));

        builder.Services.AddSingleton<IDiscoveryCacheFactory, DiscoveryCacheFactory>();

        builder.Services
            .AddClientCredentialsTokenManagement()
            .AddClient("webapi.duende.jwt", options => { })
            .AddClient("webapi.duende.sharedsecret", options => { });

        builder.Services.AddSingleton<IConfigureNamedOptions<ClientCredentialsClient>, ClientCredentialsClientConfigureOptions>();

        return builder.Build();
    }

    public static WebApplication ConfigurePipeline(this WebApplication app)
    {
        return app;
    }
}

internal class ClientCredentialsClientConfigureOptions : IConfigureNamedOptions<ClientCredentialsClient>
{
    private readonly IDiscoveryCacheFactory _discoveryCacheFactory;
    private readonly IOptions<List<HttpClientConfiguration>> _configs;

    public ClientCredentialsClientConfigureOptions(
        IDiscoveryCacheFactory discoveryCacheFactory,
        IOptions<List<HttpClientConfiguration>> configs)
    {
        _discoveryCacheFactory = discoveryCacheFactory;
        _configs = configs;
    }

    public void Configure(string? name, ClientCredentialsClient options)
    {
        if (!string.IsNullOrEmpty(name))
        {
            var config = _configs.Value.FirstOrDefault(c => c.Name == name);
            if (config == null) return;

            var disco = _discoveryCacheFactory.GetCache(name).GetAsync().GetAwaiter().GetResult();
            if (disco.IsError) throw new Exception($"Discovery failed for {name}: {disco.Error}");

            options.TokenEndpoint = disco.TokenEndpoint;
            options.ClientId = config.ClientAuthentication.ClientId;
            options.ClientSecret = config.ClientAuthentication.Secret;
            options.Scope = config.ClientAuthentication.Scope;
        }
    }

    public void Configure(ClientCredentialsClient options) => Configure("", options);
}


In [ ]:

// Build WebApplication with appsettings.json
var builder = WebApplication.CreateBuilder();
builder.Configuration.AddJsonFile("appsettings.json", optional: false, reloadOnChange: true);

var app = builder.ConfigureServices().ConfigurePipeline();
app.MapGet("/", () => "Hello from Duende + Jupyter");

// Run in background so notebook remains interactive
_ = Task.Run(() => app.RunAsync("http://localhost:5005"));
"App running on http://localhost:5005"


In [ ]:

var client = new HttpClient();
var response = await client.GetStringAsync("http://localhost:5005/");
